In [ ]:
!git clone https://github.com/anantgupta129/ERA-V1.git
!pip install torchinfo torch-lr-finder -q

In [ ]:
# !rm -rf /content/ERA-V1
%cd ERA-V1/session10

In [ ]:
import torch
import torch.optim as optim
from dataset import build_cifar
from models import CustomResNet as Net
from torchvision import transforms as T

# Load Data

In [ ]:
batch_size = 512

kwargs = {
    "batch_size": batch_size,
    "shuffle": True,
    "num_workers": 2,
    "pin_memory": True,
}

train_loader = build_cifar(set="train", **kwargs)
test_loader = build_cifar(set="test", **kwargs)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)

In [ ]:
model.summarize(device, (1, 3, 32, 32))

In [ ]:
from torch.nn import CrossEntropyLoss
from utils import find_lr

model = Net(drop=0.02).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)  #

find_lr(model, device, optimizer, criterion=CrossEntropyLoss(), dataloader=train_loader)

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR
from utils import Trainer

num_epochs = 24

scheduler = OneCycleLR(
    optimizer,
    max_lr=1.87e-02,
    steps_per_epoch=len(train_loader),
    epochs=num_epochs,
    pct_start=5 / num_epochs,
    div_factor=100,
    three_phase=False,
    final_div_factor=100,
    anneal_strategy="linear",
)

# scheduler = None
trainer = Trainer(model, device, optimizer, scheduler)
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}")
    trainer.train(train_loader)
    trainer.evaluate(test_loader)

In [ ]:
trainer.plot_history()

# Evaluate

In [ ]:
from dataset import make_transform
from utils import per_class_accuracy, plot_misclassified

test_transforms = make_transform("test")
plot_misclassified(model, test_loader, device, test_transforms)

In [ ]:
per_class_accuracy(model, device, test_loader)